In [1]:
import pandas as pd

In [22]:
pd.set_option('display.max_columns', None)
data = pd.read_csv("datasets/US_accidents.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/US_accidents.csv'

In [21]:
# Split the dataset into three parts without changing any values
# Accident Data
accident_data = data[['ID', 'Source', 'Severity', 'Start_Time', 'End_Time',
                      'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng',
                      'Distance(mi)', 'Description', 'City', 'County',
                      'State', 'Zipcode']]

# Weather Data
weather_data = data[['Start_Time', 'Weather_Timestamp', 'Temperature(F)',
                     'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
                     'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
                     'Precipitation(in)', 'Weather_Condition']]

# Road Data
road_data = data[['ID', 'Amenity', 'Bump', 'Crossing', 'Give_Way',
                  'Junction', 'No_Exit', 'Railway', 'Roundabout',
                  'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal']]

# Save datasets as CSV
accident_data.to_csv("datasets/accident_data.csv", index=False)
weather_data.to_csv("datasets/weather_data.csv", index=False)
road_data.to_csv("datasets/road_data.csv", index=False)

"/mnt/data/accident_data.csv", "/mnt/data/weather_data.csv", "/mnt/data/road_data.csv"

('/mnt/data/accident_data.csv',
 '/mnt/data/weather_data.csv',
 '/mnt/data/road_data.csv')

In [23]:
road_data

,ID,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal
0,A-2047758,False,False,False,False,False,False,False,False,False,False,False,True
1,A-4694324,False,False,False,False,False,False,False,False,False,False,False,False
2,A-5006183,False,False,False,False,False,False,False,False,False,False,False,True
3,A-4237356,False,False,False,False,False,False,False,False,False,False,False,False
4,A-6690583,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,A-6077227,False,False,False,False,False,False,False,False,False,False,False,False
499996,A-6323243,False,False,False,False,False,False,False,False,False,False,False,False
499997,A-3789256,False,False,True,True,False,False,False,False,False,False,False,True
499998,A-7030381,False,False,False,False,False,False,False,False,False,False,False,True


In [24]:
weather_data

,Start_Time,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition
0,2019-06-12 10:10:56,2019-06-12 09:53:00,77.0,77.0,62.0,29.92,10.0,NW,5.0,0.00,Fair
1,2022-12-03 23:37:14.000000000,2022-12-03 23:52:00,45.0,43.0,48.0,29.91,10.0,W,5.0,0.00,Fair
2,2022-08-20 13:13:00.000000000,2022-08-20 12:56:00,68.0,68.0,73.0,29.79,10.0,W,13.0,0.00,Fair
3,2022-02-21 17:43:04,2022-02-21 17:35:00,27.0,15.0,86.0,28.49,10.0,ENE,15.0,0.00,Wintry Mix
4,2020-12-04 01:46:00,2020-12-04 01:54:00,42.0,42.0,34.0,29.77,10.0,CALM,0.0,0.00,Fair
...,...,...,...,...,...,...,...,...,...,...,...
499995,2021-12-15 07:30:00,2021-12-15 07:14:00,40.0,32.0,77.0,29.55,10.0,SSE,15.0,0.01,Light Rain
499996,2021-12-19 16:25:00,2021-12-19 16:53:00,78.0,78.0,87.0,29.94,10.0,SSE,13.0,0.01,Partly Cloudy
499997,2022-04-13 19:28:29,2022-04-13 19:53:00,52.0,52.0,12.0,24.94,10.0,WSW,12.0,0.00,Fair
499998,2020-05-15 17:20:56,2020-05-15 17:15:00,82.0,82.0,38.0,28.70,10.0,SSW,14.0,0.00,Fair


In [25]:
accident_data

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,City,County,State,Zipcode
0,A-2047758,Source2,2,2019-06-12 10:10:56,2019-06-12 10:55:58,30.641211,-91.153481,NaN,NaN,0.000,Accident on LA-19 Baker-Zachary Hwy at Lower Z...,Zachary,East Baton Rouge,LA,70791-4610
1,A-4694324,Source1,2,2022-12-03 23:37:14.000000000,2022-12-04 01:56:53.000000000,38.990562,-77.399070,38.990037,-77.398282,0.056,Incident on FOREST RIDGE DR near PEPPERIDGE PL...,Sterling,Loudoun,VA,20164-2813
2,A-5006183,Source1,2,2022-08-20 13:13:00.000000000,2022-08-20 15:22:45.000000000,34.661189,-120.492822,34.661189,-120.492442,0.022,Accident on W Central Ave from Floradale Ave t...,Lompoc,Santa Barbara,CA,93436
3,A-4237356,Source1,2,2022-02-21 17:43:04,2022-02-21 19:43:23,43.680592,-92.993317,43.680574,-92.972223,1.054,Incident on I-90 EB near REST AREA Drive with ...,Austin,Mower,MN,55912
4,A-6690583,Source1,2,2020-12-04 01:46:00,2020-12-04 04:13:09,35.395484,-118.985176,35.395476,-118.985995,0.046,RP ADV THEY LOCATED SUSP VEH OF 20002 - 726 CR...,Bakersfield,Kern,CA,93305-2649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,A-6077227,Source1,2,2021-12-15 07:30:00,2021-12-15 07:50:30,45.522510,-123.084104,45.520225,-123.084211,0.158,Stationary traffic on OR-47 from NW Martin Rd ...,Forest Grove,Washington,OR,97116-2174
499996,A-6323243,Source1,2,2021-12-19 16:25:00,2021-12-19 17:40:37,26.702570,-80.111169,26.703141,-80.111133,0.040,Incident on MILITARY TRL near WESTGATE AVE Dri...,West Palm Beach,Palm Beach,FL,33409-4712
499997,A-3789256,Source1,2,2022-04-13 19:28:29,2022-04-13 21:33:44,34.561862,-112.259620,34.566822,-112.267150,0.549,Crash on the right shoulder on E SR-69 Northbo...,Dewey,Yavapai,AZ,86327
499998,A-7030381,Source1,3,2020-05-15 17:20:56,2020-05-15 17:50:56,38.406680,-78.619310,38.406680,-78.619310,0.000,At US-340/S Stuart Ave - Serious accident.,Elkton,Rockingham,VA,22827
